In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
cybersecurity_data = spark.read.format('csv').option('header','true').load('cybersecurity_attacks.csv')
cybersecurity_data.createOrReplaceTempView('Cybersecurity_attack')
cybersecurity_data.show()

+-----------------+----------------------+-----------+----------------+--------+-------------+-----------+------------+------------------+--------------+---------------+-----------+----------------+------------+--------------+------------------+--------------------+---------------+--------------------+-----------------+-------------+--------------+----------+
|Source IP Address|Destination IP Address|Source Port|Destination Port|Protocol|Packet Length|Packet Type|Traffic Type|Malware Indicators|Anomaly Scores|Alerts/Warnings|Attack Type|Attack Signature|Action Taken|Severity Level|  User Information|  Device Information|Network Segment|   Geo-location Data|Proxy Information|Firewall Logs|IDS/IPS Alerts|Log Source|
+-----------------+----------------------+-----------+----------------+--------+-------------+-----------+------------+------------------+--------------+---------------+-----------+----------------+------------+--------------+------------------+--------------------+----------

In [5]:
spark.sql("SELECT `Attack Type`, COUNT(*) AS Alert_Count FROM Cybersecurity_attack WHERE `Attack Type` IS NOT NULL GROUP BY `Attack Type`").show()

+-----------+-----------+
|Attack Type|Alert_Count|
+-----------+-----------+
|  Intrusion|       6597|
|       DDoS|       6785|
|    Malware|       6767|
+-----------+-----------+



In [6]:
spark.sql("SELECT `Source IP Address`, SUM(`Packet Length`) AS Total_Packet_Length FROM Cybersecurity_attack GROUP BY `Source IP Address`").show()

+-----------------+-------------------+
|Source IP Address|Total_Packet_Length|
+-----------------+-------------------+
|   46.144.162.187|              187.0|
|     18.29.92.139|              409.0|
|   205.172.254.67|              899.0|
|    141.39.56.138|               71.0|
|    45.179.10.209|             1340.0|
|  142.112.202.166|             1121.0|
|      75.170.2.44|              983.0|
|      89.72.41.50|             1308.0|
|      34.66.239.6|              484.0|
|  129.240.149.117|              305.0|
|    66.105.254.64|              551.0|
|    213.57.191.62|              462.0|
|    8.109.227.208|             1224.0|
|     81.64.181.90|              867.0|
|  164.162.114.154|             1143.0|
|   104.27.183.159|              125.0|
|     122.85.37.74|              500.0|
|     128.1.52.103|              993.0|
|  125.178.118.215|              338.0|
|   173.14.211.166|              810.0|
+-----------------+-------------------+
only showing top 20 rows



In [8]:
spark.sql("""SELECT Protocol, `Traffic Type`, AVG(`Packet Length`) AS Avg_Packet_Length FROM Cybersecurity_attack WHERE 
Protocol IS NOT NULL AND `Traffic Type` IS NOT NULL GROUP BY Protocol, `Traffic Type`""").show()

+--------+------------+-----------------+
|Protocol|Traffic Type|Avg_Packet_Length|
+--------+------------+-----------------+
|     TCP|         DNS|773.5500222321032|
|    ICMP|         DNS|796.1159747292419|
|     TCP|         FTP| 772.142793594306|
|     UDP|         FTP|783.9325206791467|
|    ICMP|         FTP|780.1052162282657|
|     UDP|         DNS|772.3678057553957|
|     TCP|        HTTP|784.6418857660925|
|     UDP|        HTTP|782.1837665298677|
|    ICMP|        HTTP|775.6669599648043|
+--------+------------+-----------------+



In [10]:
spark.sql("""SELECT `Severity Level`, Protocol, AVG(`Anomaly Scores`) AS AvgAnomalyScores FROM 
Cybersecurity_attack GROUP BY `Severity Level`, Protocol""").show();

+--------------+--------+------------------+
|Severity Level|Protocol|  AvgAnomalyScores|
+--------------+--------+------------------+
|        Medium|     UDP| 50.63631375910842|
|          High|     TCP| 49.37051716451179|
|          High|    ICMP| 50.61745614035085|
|        Medium|     TCP| 50.07901484480428|
|           Low|    ICMP| 49.48265845070423|
|          High|     UDP| 50.47782108060237|
|        Medium|    ICMP|50.222929380993925|
|           Low|     TCP|49.871998211890926|
|           Low|     UDP| 50.51781441356571|
+--------------+--------+------------------+



In [28]:
spark.sql("""
    SELECT Distinct `User Information`, `Traffic Type`, `Source IP Address` 
    FROM Cybersecurity_attack WHERE `Attack Type` = 'Malware' 
    ORDER BY `User Information`""").show()

+--------------------+------------+-----------------+
|    User Information|Traffic Type|Source IP Address|
+--------------------+------------+-----------------+
|          Aaina Arya|         FTP|      47.18.37.40|
|          Aaina Arya|        HTTP|   80.205.196.146|
|        Aaina Bakshi|         DNS|    69.253.205.98|
|          Aaina Bala|         DNS|     64.75.145.73|
|          Aaina Bala|         FTP|   163.107.130.88|
|Aaina Balasubrama...|         DNS|     98.75.202.81|
|        Aaina Bhatia|         FTP|   204.113.117.66|
|        Aaina Biswas|         FTP|  151.239.189.170|
|      Aaina Chaudhry|         DNS|   165.237.41.129|
|        Aaina Cheema|         DNS|    41.13.124.190|
|         Aaina Dalal|         FTP|  109.110.203.206|
|         Aaina Dewan|         DNS|    99.169.244.85|
|        Aaina Gulati|         FTP|     24.69.131.92|
|          Aaina Kala|         FTP|     119.45.64.73|
|         Aaina Kalla|         FTP|  201.158.180.199|
|       Aaina Khurana|      

In [35]:
spark.sql("""
    SELECT `Source IP Address`, `Severity Level`,`Attack Type`,
        CASE
            WHEN `Severity Level` = 'Low' THEN 'Green'
            WHEN `Severity Level` = 'Medium' THEN 'Yellow'
            ELSE 'Red'
        END AS Severity_Color
    FROM Cybersecurity_attack ORDER BY `Source IP Address` ASC
""").show()

+-----------------+--------------+-----------+--------------+
|Source IP Address|Severity Level|Attack Type|Severity_Color|
+-----------------+--------------+-----------+--------------+
|    1.101.138.199|        Medium|       DDoS|        Yellow|
|    1.103.203.179|           Low|       DDoS|         Green|
|    1.105.131.117|        Medium|    Malware|        Yellow|
|    1.107.181.228|          High|    Malware|           Red|
|     1.109.55.199|          High|       DDoS|           Red|
|     1.121.14.202|           Low|    Malware|         Green|
|    1.121.213.199|          High|    Malware|           Red|
|     1.123.54.243|          High|       DDoS|           Red|
|     1.127.116.39|           Low|    Malware|         Green|
|     1.131.145.70|        Medium|  Intrusion|        Yellow|
|     1.132.183.35|        Medium|    Malware|        Yellow|
|     1.137.245.81|          High|       DDoS|           Red|
|      1.141.113.8|        Medium|    Malware|        Yellow|
|     1.

In [40]:
spark.sql("""SELECT `Source IP Address`,`Traffic Type`, SUM(`Packet Length`) AS Total_Packet_Length FROM Cybersecurity_attack 
GROUP BY `Source IP Address`,`Traffic Type` ORDER BY Total_Packet_Length DESC LIMIT 10""").show()

+-----------------+------------+-------------------+
|Source IP Address|Traffic Type|Total_Packet_Length|
+-----------------+------------+-------------------+
|    59.163.53.131|         DNS|             1500.0|
|  211.180.180.240|         DNS|             1500.0|
|     21.219.2.221|        HTTP|             1500.0|
|    175.213.52.16|         DNS|             1500.0|
|   147.186.45.174|         DNS|             1500.0|
|   27.148.239.152|         FTP|             1500.0|
|     163.5.149.41|         FTP|             1500.0|
|    149.22.146.71|         FTP|             1499.0|
|       9.79.6.121|        HTTP|             1499.0|
|    165.25.19.208|         FTP|             1499.0|
+-----------------+------------+-------------------+



In [41]:
spark.sql("SELECT `Network Segment`, AVG(`Anomaly Scores`) AS Avg_Anomaly_Score FROM Cybersecurity_attack GROUP BY `Network Segment`").show()

+---------------+------------------+
|Network Segment| Avg_Anomaly_Score|
+---------------+------------------+
|      Segment A| 50.03186921559705|
|      Segment B| 50.20999252727549|
|      Segment C|50.174521625163834|
+---------------+------------------+

